In [124]:

from sympy.solvers import solve
from sympy import Symbol
import torch as t
import math

# initial guess:
x1 = Symbol('x1')
x2 = 0
x1 = solve(x1**2/4 + x2**2/5 + (x1+x2)**2/25 - 1, x1)[1]
x3 = x1+x2
print(x2,x3)
x = t.tensor([10*math.sqrt(29)/29, 0, 10*math.sqrt(29)/29])
print(x)

# define problem:
def obj(x):
    return x[0]**2+x[1]**2+x[2]**2

def h1(x):
    return x[0]**2/4+x[1]**2/5+x[2]**2/25-1

def h2(x):
    return x[0]+x[1]-x[2]

# define d and s, Choice d = x1 (x[0]); s = [x2 x3] (x[1], x[2])
def d(x):
    return t.tensor([x[0]])
def s(x):
    return t.tensor([x[1],x[2]])


# Calculate Gradient: e means partial derivative symbol
def ef_ed(x):
    return t.tensor([2*x[0]])

def ef_es(x):
    return t.tensor([2*x[1],2*x[2]])

def eh_es(x):
    return t.tensor([[2*x[1]/5,2*x[2]/25],
                    [1,-1]])

def eh_ed(x):
    return t.tensor([[x[0]/2],
                     [1]])

def df_dd(x):
    return ef_ed(x) - t.matmul(t.matmul(ef_es(x),t.inverse(eh_es(x))),eh_ed(x))

def linesearch(x):

    a = 1
    b = 0.5
    c = 0.3
    i = 0
    def f_a(x):
        x_k_1 = t.cat([d(x)-a*df_dd(x), t.squeeze(s(x) + a*t.transpose(t.matmul(t.inverse(eh_es(x)),eh_ed(x))*df_dd(x),0,1))])
        return obj(x_k_1)
    def f__a(x):
        return obj(x) - a*c*df_dd(x)**2

    while(f_a(x)>f__a(x)):
        a = b*a
        i = i+1
    print(f_a(x))
    print(f__a(x))
    print(i)
    print(a)

linesearch(x)

0 10*sqrt(29)/29
tensor([1.8570, 0.0000, 1.8570])
tensor(5.9488)
tensor([6.4510])
8
0.00390625


In [45]:
import torch as t


tensor([0.0000, 1.8570])
